In [1]:
import os
os.chdir('/camp/home/wilkino/home/POSTDOC/borzoi/borzoi/examples/')

In [2]:
import json
import os
import time
import warnings

import h5py
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import pysam
import pyfaidx
import tensorflow as tf

from baskerville import seqnn
from baskerville import gene as bgene
from baskerville import dna

from borzoi_helpers import *

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

2024-06-28 18:58:58.316025: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-28 18:58:58.316103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-28 18:58:58.447913: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-28 18:58:58.647481: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-28 18:59:09.260809: W tensorflow/comp

In [ ]:
# %%bash

# #Download model weights
# for fold in f0 f1 f2 f3; do
#   mkdir -p "saved_models/$fold/"
#   local_model="saved_models/$fold/model0_best.h5"
#   if [ -f "$local_model" ]; then
#     echo "$fold model already exists."
#   else
#     wget --progress=bar:force "https://storage.googleapis.com/seqnn-share/borzoi/$fold/model0_best.h5" -O "$local_model"
#   fi
# done

# #Download and uncompress annotation files
# if [ -f gencode41_basic_nort.gtf ]; then
#   echo "Annotation already exists."
# else
#   wget -O - https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_nort.gtf.gz | gunzip -c > gencode41_basic_nort.gtf
# fi
# if [ -f gencode41_basic_protein_splice.csv.gz ]; then
#   echo "Splice sites already exist."
# else
#   wget https://storage.googleapis.com/seqnn-share/helper/gencode41_basic_protein_splice.csv.gz
# fi
# if [ -f polyadb_human_v3.csv.gz ]; then
#   echo "PolyA sites already exist."
# else
#   wget https://storage.googleapis.com/seqnn-share/helper/polyadb_human_v3.csv.gz
# fi

# #Download and index hg38 genome
# if [ -f hg38.fa ]; then
#   echo "Human genome FASTA already exists."
# else
#   wget -O - http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz | gunzip -c > hg38.fa
# fi

In [ ]:
# pyfaidx.Faidx('hg38.fa')

In [3]:
#Model configuration

params_file = 'params_pred.json'
targets_file = 'targets_gtex.txt' #Subset of targets_human.txt

seq_len = 524288
n_folds = 1       #To use only one model fold, set to 'n_folds = 1'. To use all four folds, set 'n_folds = 4'.
rc = True         #Average across reverse-complement prediction

#Read model parameters

with open(params_file) as params_open :
    
    params = json.load(params_open)
    
    params_model = params['model']
    params_train = params['train']

#Read targets

targets_df = pd.read_csv(targets_file, index_col=0, sep='\t')
target_index = targets_df.index

#Create local index of strand_pair (relative to sliced targets)
if rc :
    strand_pair = targets_df.strand_pair
    
    target_slice_dict = {ix : i for i, ix in enumerate(target_index.values.tolist())}
    slice_pair = np.array([
        target_slice_dict[ix] if ix in target_slice_dict else ix for ix in strand_pair.values.tolist()
    ], dtype='int32')

#Initialize model ensemble

models = []
for fold_ix in range(n_folds) :
    
    model_file = "saved_models/f" + str(fold_ix) + "/model0_best.h5"

    seqnn_model = seqnn.SeqNN(params_model)
    seqnn_model.restore(model_file, 0)
    seqnn_model.build_slice(target_index)
    if rc :
        seqnn_model.strand_pair.append(slice_pair)
    seqnn_model.build_ensemble(rc, [0])
    
    models.append(seqnn_model)


2024-06-28 18:59:23.040412: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31141 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:1a:00.0, compute capability: 7.0


In [4]:
#Initialize fasta sequence extractor

fasta_open = pysam.Fastafile('hg38.fa')

#Load splice site annotation

splice_df = pd.read_csv('gencode41_basic_protein_splice.csv.gz', sep='\t', compression='gzip')

print("len(splice_df) = " + str(len(splice_df)))


len(splice_df) = 404837


In [5]:
#Load GTF (optional; needed to compute exon coverage attributions for example gene)

transcriptome = bgene.Transcriptome('gencode41_basic_nort.gtf')

In [6]:
search_gene = 'ENSG00000144285.23'
center_pos = 166_060_000
chrom = 'chr2'

start = center_pos - seq_len // 2
end = center_pos + seq_len // 2

print(start)

#Get exon bin range
gene_keys = [gene_key for gene_key in transcriptome.genes.keys() if search_gene in gene_key]

print(gene_keys)

gene = transcriptome.genes[gene_keys[0]]

#Determine output sequence start
seq_out_start = start + seqnn_model.model_strides[0]*seqnn_model.target_crops[0]
seq_out_len = seqnn_model.model_strides[0]*seqnn_model.target_lengths[0]

print(seq_out_start)
print(seq_out_len)

#Determine output positions of gene exons
gene_slice = gene.output_slice(seq_out_start, seq_out_len, seqnn_model.model_strides[0], False)


165797856
['ENSG00000144285.23']
165798368
523264


In [7]:
#Print index of GTEx blood and muscle tracks in targets file

targets_df['local_index'] = np.arange(len(targets_df))

print("blood tracks = " + str(targets_df.loc[targets_df['description'] == 'RNA:blood']['local_index'].tolist()))
print("muscle tracks = " + str(targets_df.loc[targets_df['description'] == 'RNA:muscle']['local_index'].tolist()))
print("brain tracks = " + str(targets_df.loc[targets_df['description'] == 'RNA:brain']['local_index'].tolist()))


blood tracks = [9, 10, 11]
muscle tracks = [47, 48, 49]
brain tracks = [17, 18, 19]


In [ ]:
print(list(targets_df['description']))

In [7]:
def one_hot_mut(sequence_one_hot_wt, poses, alts, start):
    sequence_one_hot_mut = np.copy(sequence_one_hot_wt)
    
    for pos, alt in zip(poses, alts):
        alt_ix = -1
        if alt == 'A' :
            alt_ix = 0
        elif alt == 'C' :
            alt_ix = 1
        elif alt == 'G' :
            alt_ix = 2
        elif alt == 'T' :
            alt_ix = 3

        sequence_one_hot_mut[pos-start-1] = 0.
        sequence_one_hot_mut[pos-start-1, alt_ix] = 1.
                                   
    return sequence_one_hot_mut



In [ ]:
best_single_muts = pd.read_csv("/camp/home/wilkino/home/POSTDOC/borzoi/borzoi/jenna/best_single_muts.csv")

best_single_muts = best_single_muts.iloc[::-1].reset_index(drop=True)

start_trim = 169000 // 32
end_trim = (166175000 - start) // 32
window = 20 # window in which to do second mutation

# Specify the output CSV file path
csv_file = "/camp/home/wilkino/home/POSTDOC/borzoi/borzoi/jenna/ism_results_second_mut.csv"

# Open the CSV file in append mode to add data
with open(csv_file, mode='w') as file:
    
    file.write('position,nucleotide,score,first_mutation\n')

    # First get unmutated sequence
    sequence_one_hot_wt = process_sequence(fasta_open, chrom, start, end)
    y_wt = predict_tracks(models, sequence_one_hot_wt)
    brain_wt = 100000 * y_wt[:, :, start_trim:end_trim, 17:20]
    wt_sum = int(tf.reduce_sum(brain_wt)) / 100000
    
    file.write('-,-,' + str(wt_sum) + ',-\n')
    
yo = 0

for p, mut in zip(best_single_muts['position'], best_single_muts['nucleotide']):
    
    sequence_one_hot_1mut = one_hot_mut(sequence_one_hot_wt, [p], [mut], start)
    
    for offset in range(-window, window):
        if offset == 0:
            continue
            
        print(offset)
        
        if p + offset - start < 0:
            continue
        
        if p + offset - start > seq_len:
            continue
        
        for new_nt in ['A', 'C', 'T', 'G']:
            sequence_one_hot_2mut = one_hot_mut(sequence_one_hot_1mut, [p+offset], [new_nt], start)
            
            if tf.reduce_all(tf.equal(sequence_one_hot_1mut, sequence_one_hot_2mut)):
                continue


            y_mut = predict_tracks(models, sequence_one_hot_2mut)
            brain_mut = 100000 * y_mut[:, :, start_trim:end_trim, 17:20]
            mut_sum = int(tf.reduce_sum(brain_mut)) / 100000

            with open(csv_file, 'a') as file:
                file.write(str(p+offset) + ',' + new_nt + ',' + str(mut_sum) + ',' + str(p) + ';' + mut + '\n')
            
        yo += 1
    

    



2024-06-28 19:00:02.521396: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
1
2
3
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
1
2
3
4
5
6
7
8
9
10
12
13
14
15
16

In [26]:


# # Read the best_targets.csv file
# best_targets = pd.read_csv("/camp/home/wilkino/home/POSTDOC/borzoi/borzoi/jenna/best_targets.csv")

# start_trim = 169000 // 32
# end_trim = (166175000 - start) // 32

# # Specify the output CSV file path
# csv_file = "/camp/home/wilkino/home/POSTDOC/borzoi/borzoi/jenna/ism_results.csv"

# # Open the CSV file in append mode to add data
# with open(csv_file, mode='w') as file:
    
#     file.write('position,nucleotide,score\n')

#     # First get unmutated sequence
#     sequence_one_hot_wt = process_sequence(fasta_open, chrom, start, end)
#     y_wt = predict_tracks(models, sequence_one_hot_wt)
#     brain_wt = 100000 * y_wt[:, :, start_trim:end_trim, 17:20]
#     wt_sum = int(tf.reduce_sum(brain_wt)) / 100000
    
#     file.write('-,-,' + str(wt_sum) + '\n')
    

# yo = 0


# # Now go through all possible mutants
# for p in best_targets['pos']:
#     for new_nt in ['A', 'C', 'G', 'T']:
#         sequence_one_hot_mut = one_hot_mut(sequence_one_hot_wt, [p], [new_nt], start)

#         if tf.reduce_all(tf.equal(sequence_one_hot_wt, sequence_one_hot_mut)):
#             continue

#         y_mut = predict_tracks(models, sequence_one_hot_mut)
#         brain_mut = 100000 * y_mut[:, :, start_trim:end_trim, 17:20]
#         mut_sum = int(tf.reduce_sum(brain_mut)) / 100000
        
#         with open(csv_file, 'a') as file:
#             file.write(str(p) + ',' + new_nt + ',' + str(mut_sum) + '\n')
            
#         yo += 1


In [11]:
best_single_muts = pd.read_csv("/camp/home/wilkino/home/POSTDOC/borzoi/borzoi/jenna/best_single_muts.csv")
print(best_single_muts)

     position nucleotide       score  downsampled
0   165991702          A  1588.90755       165990
1   165991705          C  1586.05150       165990
2   165991706          A  1587.59867       165990
3   165991706          T  1587.05882       165990
4   166091182          A  1615.19521       166090
5   166127381          T  1616.44106       166120
6   166127454          C  1586.54045       166120
7   166127456          A  1586.23943       166120
8   166127493          G  1596.53002       166120
9   166127502          C  1612.17347       166120
10  166127560          G  1605.82294       166120
11  166127876          T  1590.35662       166120
12  166127878          C  1599.54436       166120
13  166127880          C  1611.09994       166120
14  166127881          C  1599.09431       166120
15  166149042          C  1618.76229       166140
16  166149048          T  1586.16778       166140


In [ ]:
for p in best_targets['pos']:
    print(p)
    break

In [ ]:
%%time
#Predict for chr10_116952944_T_C
# (~6 minutes on CPU w 1 fold; ~2 minutes on GPU)

save_figs = False
save_suffix = '_meh_'

sequence_one_hot_wt = process_sequence(fasta_open, chrom, start, end)

#Induce mutation(s)
sequence_one_hot_mut = np.copy(sequence_one_hot_wt)

for pos, alt in zip(poses, alts) :
    alt_ix = -1
    if alt == 'A' :
        alt_ix = 0
    elif alt == 'C' :
        alt_ix = 1
    elif alt == 'G' :
        alt_ix = 2
    elif alt == 'T' :
        alt_ix = 3

    sequence_one_hot_mut[pos-start-1] = 0.
    sequence_one_hot_mut[pos-start-1, alt_ix] = 1.

#Make predictions
y_wt = predict_tracks(models, sequence_one_hot_wt)
y_mut = predict_tracks(models, sequence_one_hot_mut)

print('done')



brain_wt = y_wt[:, :, :, 17:20]
brain_mut = y_mut[:, :, :, 17:20]
print(tf.reduce_sum(brain_wt))
print(tf.reduce_sum(brain_mut))


In [ ]:
#Visualize quantized tracks over SNP

plot_window = 131072*2
bin_size = 32
pad = 16

rescale_tracks = True
normalize_counts = False

anno_df = None #splice_df

#Tracks
track_indices = [
    np.arange(0, 89).tolist(),
    [9, 10, 11],
    [47, 48, 49],
    [17, 18, 19],
]

track_names = [
    'GTEx Coverage (All tissues)',
    'GTEx Coverage (Blood)',
    'GTEx Coverage (Muscle)',
    'Brain',
]

track_scales = [0.01]*4
track_transforms = [3./4.]*4
soft_clips = [384.]*4

print("-- Counts --")
plot_coverage_track_pair_bins(
    y_wt,
    y_mut,
    chrom,
    start,
    center_pos,
    poses,
    track_indices,
    track_names,
    track_scales,
    track_transforms,
    soft_clips,
    plot_window=plot_window,
    normalize_window=1 * plot_window,
    bin_size=bin_size,
    pad=pad,
    rescale_tracks=rescale_tracks,
    normalize_counts=normalize_counts,
    save_figs=save_figs,
    save_suffix=save_suffix,
    gene_slice=gene_slice,
    anno_df=anno_df,
)

In [ ]:

%%time
#Get contribution scores (gradient) for brain GTEX tracks
# (~20 minutes on CPU; otherwise runnable only on 40GB GPU cards, e.g. A100)

_, _, [pred_grad_wt] = get_prediction_gradient_w_rc(
    models,
    [sequence_one_hot_wt],
    prox_bin_start=0,
    prox_bin_end=1,
    dist_bin_start=0,
    dist_bin_end=1,
    track_index=target_index[[17, 18, 19]].tolist(),
    track_scale=0.01,
    track_transform=3./4.,
    clip_soft=384.,
    dist_bin_index=gene_slice.tolist(),
    use_mean=False,
    use_ratio=False,
    use_logodds=False,
    subtract_avg=True,
    fold_index=np.arange(n_folds).tolist(),
)


In [ ]:
import pickle

# Assume pred_grad_wt is already defined
# Replace this line with the actual call to your function that defines pred_grad_wt
# _, _, [pred_grad_wt, pred_grad_mut] = get_prediction_gradient_w_rc(...)

# Save the pred_grad_wt using pickle
with open('pred_grad_wt.pkl', 'wb') as f:
    pickle.dump(pred_grad_wt, f)

print("pred_grad_wt has been saved to 'pred_grad_wt.pkl'")


In [ ]:
#Visualize gradient contribution scores (128 bp window centered on variant)

visualize_input_gradient_pair(
    pred_grad_wt,
    pred_grad_mut,
    plot_start=(poses[0] - start) - 64,
    plot_end=(poses[0] - start) + 64,
    save_figs=False,
    fig_name=chrom + '_' + str(poses[0]) + '_prediction_grad_gtex_snp_4_folds_gtex_blood_cov_undo_clip'
)


In [ ]:
%%time
#Get contribution scores (ISM) for blood GTEX tracks
# (not feasible to run on CPU; ~33 minutes on GPU)

[pred_ism_wt, pred_ism_mut] = get_ism(
    models,
    [sequence_one_hot_wt, sequence_one_hot_mut],
    ism_start=(poses[0] - start) - 64,
    ism_end=(poses[0] - start) + 64,
    prox_bin_start=0,
    prox_bin_end=1,
    dist_bin_start=0,
    dist_bin_end=1,
    track_index=[9, 10, 11],
    track_scale=0.01,
    track_transform=3./4.,
    clip_soft=384.,
    dist_bin_index=gene_slice.tolist(),
    use_mean=True,
    use_ratio=False,
    use_logodds=False,
)


In [ ]:
#Visualize ISM contribution scores (128 bp window centered on variant)

visualize_input_gradient_pair(
    pred_ism_wt,
    pred_ism_mut,
    plot_start=(poses[0] - start) - 64,
    plot_end=(poses[0] - start) + 64,
    save_figs=False,
    fig_name=chrom + '_' + str(poses[0]) + '_prediction_ism_gtex_snp_4_folds_gtex_blood_cov_undo_clip'
)


In [ ]:
%%time
#Get contribution scores (ISM shuffle) for blood GTEX tracks
# (not feasible to run on CPU; ~135 minutes on GPU)

[pred_ism_wt, pred_ism_mut] = get_ism_shuffle(
    models,
    [sequence_one_hot_wt, sequence_one_hot_mut],
    ism_start=(poses[0] - start) - 64,
    ism_end=(poses[0] - start) + 64,
    prox_bin_start=0,
    prox_bin_end=1,
    dist_bin_start=0,
    dist_bin_end=1,
    track_index=[9, 10, 11],
    track_scale=0.01,
    track_transform=3./4.,
    clip_soft=384.,
    window_size=5,
    n_samples=12,
    dist_bin_index=gene_slice.tolist(),
    use_mean=True,
    use_ratio=False,
    use_logodds=False,
)


In [ ]:
#Visualize ISM Shuffle contribution scores (128 bp window centered on variant)

visualize_input_gradient_pair(
    pred_ism_wt,
    pred_ism_mut,
    plot_start=(poses[0] - start) - 64,
    plot_end=(poses[0] - start) + 64,
    save_figs=False,
    fig_name=chrom + '_' + str(poses[0]) + '_prediction_ism_shuffle_gtex_snp_4_folds_gtex_blood_cov_undo_clip'
)


In [ ]:
#Load samples with reference- and alternate alleles respectively

#These files are protected by dbGaP - email to request access

cov_files_wt = pd.read_csv("gtex_ref_chr10_116952944_T_C.txt", sep='\t', names=['file'])['file'].values.tolist()
cov_files_mut = pd.read_csv("gtex_alt_chr10_116952944_T_C.txt", sep='\t', names=['file'])['file'].values.tolist()


In [ ]:
#Visualize measured coverage tracks for chr10_116952944_T_C

save_figs = False
save_suffix = '_chr10_116952944_T_C_meas_32_subjects'

center_pos = 116952944

chrom = 'chr10'
poses = [116952944]
alts = ['C']

start = center_pos - seq_len // 2
end = center_pos + seq_len // 2

blacklist_bed = "/home/drk/common/data/genomes/hg38/blacklist/blacklist_hg38_all.bed"

read_coverage_func_wt, close_coverage_func_wt = get_coverage_reader(cov_files_wt, 16384, 16, blacklist_bed)
read_coverage_func_mut, close_coverage_func_mut = get_coverage_reader(cov_files_mut, 16384, 16, blacklist_bed)

gtex_targets_wt = read_coverage_func_wt(chrom, start, end, clip_soft=384., clip=768., scale=0.01)
gtex_targets_mut = read_coverage_func_mut(chrom, start, end, clip_soft=384., clip=768., scale=0.01)

close_coverage_func_wt()
close_coverage_func_mut()

print("gtex_targets_wt.shape = " + str(gtex_targets_wt.shape))
print("gtex_targets_mut.shape = " + str(gtex_targets_mut.shape))

plot_window = 131072
bin_size = 32
pad = 16

anno_df = None #splice_df

rescale_tracks = True
normalize_counts = True

#Tracks
track_indices = [
    np.arange(32).tolist(),
]

track_names = [
    'GTEx Blood (32 subjects)',
]

track_scales = [
    0.01,
]

track_transforms = [
    3./4.,
]

soft_clips = [
    384.,
]

print("-- Counts --")
plot_coverage_track_pair_bins(
    gtex_targets_wt[None, None, ...],
    gtex_targets_mut[None, None, ...],
    chrom,
    start,
    center_pos,
    poses,
    track_indices,
    track_names,
    track_scales,
    track_transforms,
    soft_clips,
    plot_window=plot_window,
    normalize_window=4 * plot_window,
    bin_size=bin_size,
    pad=pad,
    rescale_tracks=rescale_tracks,
    normalize_counts=normalize_counts,
    save_figs=save_figs,
    save_suffix=save_suffix,
    gene_slice=gene_slice,
    anno_df=anno_df,
)
